In [10]:
import torch
import torchvision 
from torchvision.datasets import MNIST
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torch.utils.data import random_split
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.nn as nn 
import torch.optim as optim

In [2]:
# downloaded dataset from the MINST dataset
dataset = MNIST(root="data/", download=True)


In [33]:
class ExactMNISTClassifier(nn.Module):
     def __init__(self):
        super(ExactMNISTClassifier, self).__init__()

        self.activation = nn.ReLU()
        

        self.layer1 = nn.Linear(784, 1028)    
        self.layer2 = nn.Linear(1028, 512)   
        self.layer3 = nn.Linear(512, 256)     
        self.layer4 = nn.Linear(256, 128)     
        self.layer5 = nn.Linear(128, 64)     
        self.layer6 = nn.Linear(64, 32)       
        self.output_layer = nn.Linear(32, 10) 
        
        
        self.dropout = nn.Dropout(0.3)
        
  
        self._initialize_weights()
        


     def _initialize_weights(self):
        for layer in [self.layer1, self.layer2, self.layer3, 
                     self.layer4, self.layer5, self.layer6, self.output_layer]:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_uniform_(layer.weight)
                if layer.bias is not None:
                    nn.init.zeros_(layer.bias)

     def forward(self, x):
        # Flatten input
        x = x.view(-1, 28*28)


        x = self.activation(self.layer1(x))
        x = self.dropout(x)

        x = self.activation(self.layer2(x))
        x = self.dropout(x)

        x = self.activation(self.layer3(x))
        x = self.dropout(x)

        x = self.activation(self.layer4(x))
        x = self.dropout(x)

        x = self.activation(self.layer5(x))
        x = self.dropout(x)

        x = self.activation(self.layer6(x))
        x = self.dropout(x)

        x = self.output_layer(x)

        return x
     



In [24]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = MNIST(
    root="./data",
    train=True,
    download=True,
    transform=transform
)

test_dataset = MNIST(
    root="./data",
    train=False,
    download=True,
    transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ExactMNISTClassifier().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [26]:
def train(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)


In [27]:
def test(model, loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return 100 * correct / total

In [34]:
for epoch in range(10):
    train_loss = train(model, train_loader, optimizer, criterion)
    accuracy = test(model, test_loader)

    print(f"Epoch [{epoch+1}/10], Loss: {train_loss:.4f}, Accuracy: {accuracy:.2f}%")


AttributeError: 'ExactMNISTClassifier' object has no attribute 'activation'

In [30]:
def identify_digit(model, image_tensor):
    """
    image_tensor: shape [1, 1, 28, 28]
    """
    model.eval()
    with torch.no_grad():
        image_tensor = image_tensor.to(device)
        output = model(image_tensor)
        predicted_digit = torch.argmax(output, dim=1).item()

    return predicted_digit


In [21]:
image, label = test_dataset[0]
image = image.unsqueeze(0)  # add batch dimension

prediction = identify_digit(model, image)

print("Predicted:", prediction)
print("Actual:", label)


Predicted: 7
Actual: 7
